In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn import datasets
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

In [ ]:
def MapCatToInt(training_df):
    training_df['iFOB'] = training_df['FOB']

    cleanup_nums = {"iFOB": {'Out-Patient': 1, 'In-Patient': 2, 'Dental': 3}}
    training_df = training_df.replace(cleanup_nums, method='bfill')
    print(training_df["iFOB"].value_counts())
    print()
    print(training_df["PROVIDER_TYPE"].value_counts())
    training_df['iPROVIDER_TYPE'] = training_df['PROVIDER_TYPE']

    Provider_Type_Map = {'iPROVIDER_TYPE':
                             {'Hospital': 1, 'Polyclinic - Group of Doctors with lab, X-ray, treatment facilities': 2,
                              'Pharmacy': 3, 'Diagnostic Center': 4, 'Laboratory': 5, 'Dental Center': 6,
                              'Radiology Center': 7, 'Physiotherapy Center': 8
                              }

                         }
    training_df = training_df.replace(Provider_Type_Map, method='bfill')
    print()
    print(training_df["iPROVIDER_TYPE"].value_counts())

    training_df['iClaimsValueCat'] = training_df['NET_CLAIM_VALUE']

    training_df.loc[training_df['NET_CLAIM_VALUE'] <= 500, 'iClaimsValueCat'] = 'Below 500'
    training_df.loc[(training_df['NET_CLAIM_VALUE'] > 500) & (
                training_df['NET_CLAIM_VALUE'] <= 5000), 'iClaimsValueCat'] = '501-5000'
    training_df.loc[(training_df['NET_CLAIM_VALUE'] > 5000) & (
                training_df['NET_CLAIM_VALUE'] <= 10000), 'iClaimsValueCat'] = '5001 - 10000'
    training_df.loc[(training_df['NET_CLAIM_VALUE'] > 10000) & (
                training_df['NET_CLAIM_VALUE'] <= 15000), 'iClaimsValueCat'] = '10001 - 15000'
    training_df.loc[(training_df['NET_CLAIM_VALUE'] > 15000) & (
                training_df['NET_CLAIM_VALUE'] <= 20000), 'iClaimsValueCat'] = '15001 - 20000'
    training_df.loc[(training_df['NET_CLAIM_VALUE'] > 20000) & (
                training_df['NET_CLAIM_VALUE'] <= 25000), 'iClaimsValueCat'] = '20001 - 25000'
    training_df.loc[(training_df['NET_CLAIM_VALUE'] > 25000) & (
                training_df['NET_CLAIM_VALUE'] <= 30000), 'iClaimsValueCat'] = '25001 - 30000'
    training_df.loc[training_df['NET_CLAIM_VALUE'] > 30000, 'iClaimsValueCat'] = 'Above 30000'

    claimsValueMap = {
        'iClaimsValueCat': {'Below 500': 1, '501-5000': 2, '5001 - 10000': 3, '10001 - 15000': 4, '15001 - 20000': 5,
                            '20001 - 25000': 6, '25001 - 30000': 7, 'Above 30000': 8
                            }
        }
    print()
    training_df = training_df.replace(claimsValueMap, method='bfill')
    training_df['POLICY_START_YEAR'] = pd.DatetimeIndex(training_df['POLICY_INCEPTION_DATE']).year
    return training_df

In [ ]:
AfiyaCliams = pd.read_csv('../input/GM-INSIS-AAFIY-CLAIMS-CORE.csv',low_memory=False)

In [ ]:
columns = ['MEMBER_ID_NO','CLIENT_GROUP', 'CLIENT_NAME', 'POLICY_NUMBER','POLICY_INCEPTION_DATE','DATE_OF_BIRTH','GENDER','NATIONALITY','PROVIDER_NAME'
           ,'PROVIDER_TYPE','PROVIDER_CITY','ATTENDING_DOCTOR_NAME','SPECIALITY','CLAIM_NO','TREATMENT_DATE','CURRENCY','FINAL_AMT','POLICY_ID']

AfiyaCliams = AfiyaCliams[columns]

In [ ]:
AfiyaCliams['POLICY_INCEPTION_DATE'] = pd.to_datetime(AfiyaCliams['POLICY_INCEPTION_DATE'])
AfiyaCliams['DATE_OF_BIRTH'] = pd.to_datetime(AfiyaCliams['DATE_OF_BIRTH'])
AfiyaCliams['TREATMENT_DATE'] = pd.to_datetime(AfiyaCliams['TREATMENT_DATE'])
AfiyaCliams.POLICY_ID.fillna(0,inplace=True)
AfiyaCliams['POLICY_ID'] = AfiyaCliams.POLICY_ID.astype(int) 
AfiyaCliams.CLAIM_NO.fillna(0,inplace=True)
AfiyaCliams.CLAIM_NO = AfiyaCliams.CLAIM_NO.astype(int) 
AfiyaCliams.rename(columns={'MEMBER_ID_NO':'CARD_NUMBER'},inplace=True)
AfiyaCliams['POLICY_START_YEAR'] = pd.DatetimeIndex(AfiyaCliams.POLICY_INCEPTION_DATE).year
AfiyaCliams['DOB_YEAR'] = pd.DatetimeIndex(AfiyaCliams.DATE_OF_BIRTH).year
AfiyaCliams['TREATMENT_WEEKDAY'] = AfiyaCliams.DATE_OF_BIRTH.dt.weekday_name
AfiyaCliams.dtypes

In [ ]:
AfiyaCliams['CURRENCY'].value_counts(dropna=False)